## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


    
### Оглавление
1. [Шаг 1. Откроем файл с данными и изучим общую информацию](#step1)  
2. [Шаг 2. Предобработка пропусков:](#step2)<br>
 2.1. [Исследуем пропуски в таблице и заменим наиболее релевантными значениями](#step2-1)<br>
 2.2. [Приведем типы данных к целочисленным](#step2-2)<br> 
 2.3. [Изучим и обработаем дубликаты](#step2-3)<br>
 2.4. [Лемматизация причин кредита](#step2-4)<br>
 2.5. [Категоризация словарей для дальнейшего анализа и ответа на вопросы](#step2-5)<br>
3. [Ответы на вопросы](#step3)<br>
4. [Итоговый вывод](#step4)

### Шаг 1. Открываем файл с данными и изучаем общую информацию.  <a id="step1"></a> 

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

df = pd.read_csv('./datasets/credits.csv')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В таблице есть поля типа int, float и object. 
Всего в таблице 21 525 строк и 12 столбцов. В столбцах days_employed и total_income по 19 351 - в них есть пропуски.
Столбцы названы корректно.

##### Вопросы:
- Почему возникли пропуски? 
- Как будем их заполнять? 
- Что с типами данных, они корректные?

### Шаг 2. Предобработка данных <a id="step2"></a> 

### Обработка пропусков <a id="step2-1"></a> 

In [2]:
print('Кол-во строк в таблице:', df.shape[0])
print('Кол-во пропусков в столбце days_employed:', df['days_employed'].isna().sum())
print('Процент пропусков: {:.1%}'.format(df['days_employed'].isna().sum() / df.shape[0]))

#Убираю отрицательные значения из days_employed
df['days_employed'] = df['days_employed'].abs()

#Делаю вывод среднего и медианы
print('Среднее:', df['days_employed'].mean())
print('Медиана:', df['days_employed'].median())

#Заполняю пропуски методом .loc[]
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median()


df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

Кол-во строк в таблице: 21525
Кол-во пропусков в столбце days_employed: 2174
Процент пропусков: 10.1%
Среднее: 66914.72890682195
Медиана: 2194.220566878695


In [3]:
print('Кол-во пропусков в столбце total_income:', df['total_income'].isna().sum())
print('')
#Убираю отрицательные значения из total_income
df['total_income'] = df['total_income'].abs()
print('Среднее:', df['total_income'].mean())
print('Медиана:', df['total_income'].median())

df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median()

Кол-во пропусков в столбце total_income: 2174

Среднее: 167422.3022081719
Медиана: 145017.93753253992


### Вывод

Нашел пропуски в строках с рабочим стажем и доходом. Просмотрев несколько десятков пропущенных значений пришел к выводу - это полностью случайные пропуски. Нет какой-либо зависимости с другими столбцами и неизвестно как восстановить какими-либо значениями.

Всего 2174 пропуска. Это 10.1% - много.

Заменил минусовые значения трудового стажа функцией abs(). Заполнил пропуски медианным значением.

Заполнил пропуски с использованием метода .loc - вроде бы понимаю как работает, но не осознал еще.

Дополнительно заменил -1 на 1 и 20 на 2 в столбце детей.

### Замена типа данных <a id="step2-2"></a> 

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

### Вывод

Заменил в столбцах days_employed и total_income тип на int. Для замены использовал метод astype('int').

### Обработка дубликатов <a id="step2-3"></a> 

In [5]:
print(df.groupby('education')['education'].count())
df['education'] = df['education'].str.lower()


print(df.groupby('family_status')['family_status'].count())
df['family_status'] = df['family_status'].str.lower()


df['purpose'] = df['purpose'].str.lower()
print(df.groupby('purpose')['purpose'].count())


print(df.groupby('gender')['gender'].count())
df = df.drop(df.loc[df['gender'] == 'XNA', 'gender'].index)

education
ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64
family_status
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64
purpose
автомобили                                478
автомобиль                                495
высшее образование                        453
дополнительное образование                462
жилье                                     647
заняться высшим образованием              496
заняться образованием                 

In [6]:
print('Дубликатов', df.duplicated().sum())
df.loc[df.duplicated()]

Дубликатов 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,2194,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,2194,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,2194,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


### Вывод

Привел все строки к нижнему регистру и удалил дубликаты методом drop_duplicates(). 

Привел значения в столбцах education, family_status и purpose к нижнему регистру - консистентность значений этих столбцов стала нормальной. До этого был разброс по типу написания значений, хотя в соседних столбцах education_id и family_status_id айдишники стояли одинаковые для разного написания.

После проверил пол на уникальные значения и нашел значение XNA. Удалил его методом drop(), потому что не ясно к какому полу соотнести.

Следующий этап - проверка на дубликаты. Показывает 71. Исследовал их и пришел к выводу, что это значения с одинаковым days_employed и total_income, а все остальные значения разные.

    
##### Вопросы
- Сколько у нас было дубликатов? 
- Если удалить дубликаты, потеряются ли ценные данные?
- Откуда у нас возникли дубликаты? Можно ли их удалять?

### Лемматизация <a id="step2-4"></a> 

In [7]:
def lemmatize_purposes(row):
    lemmas = m.lemmatize(row['purpose'])
    #print(lemmas)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'займ на недвижимость'
    
    elif 'автомобиль' in lemmas:
        return 'займ на автомобиль'
    
    elif 'образование' in lemmas:
        return 'займ на образование'
    
    elif 'свадьба' in lemmas:
        return 'займ на свадьбу'

df['purpose_lemma'] = df.apply(lemmatize_purposes, axis = 1)

print(df['purpose_lemma'].value_counts())

займ на недвижимость    10839
займ на автомобиль       4315
займ на образование      4022
займ на свадьбу          2348
Name: purpose_lemma, dtype: int64


In [8]:
data = df['purpose'].unique()
columns = ['purpose']

purposes_dict = pd.DataFrame(data = data, columns = columns)

purposes_dict['lemmas_purpose'] = purposes_dict.apply(lemmatize_purposes, axis = 1)
#print(purposes_dict)

df.merge(purposes_dict, on = 'purpose', how = 'left')
print(df['purpose_lemma'].value_counts())

займ на недвижимость    10839
займ на автомобиль       4315
займ на образование      4022
займ на свадьбу          2348
Name: purpose_lemma, dtype: int64


### Вывод

Использовал библиотеку pymystem3 для лемматизации целей кредита. 
Сделал функцию для преобразования разных форм целей кредита к единому виду при помощи лемматизации. Добавил это все в отдельный столбец в датафрейме и вывел количество целей кредита.

Самые популярные:
1. кредиты на жилье
2. авто\образование
3. свадьба

**PS: сделал двумя способами для практики. Результат одинаковый.**

### Категоризация данных <a id="step2-5"></a> 

In [ ]:
total_income_median = df['total_income'].median()
total_income_25 = df['total_income'].mean() - (df['total_income'].mean() * 0.5)
total_income_75 = df['total_income'].mean() + (df['total_income'].mean() * 0.5)

def check_children(row):
    
    if row['children'] > 0:
        result = 'есть дети'

    else:
        result = 'нет детей'

    return result


def check_family_status(row):
    
    if row['family_status_id'] in (0, 1):
        result = 'семейный'

    else:
        result = 'холост'

    return result


def check_total_income(row):   
    
    if row['total_income'] <= total_income_25:
        result = '0 - 25%'
    
    elif row['total_income'] > total_income_25 and row['total_income'] <= total_income_median:
        result = '25% - 50%'
    
    elif row['total_income'] > total_income_median and row['total_income'] <= total_income_75:
        result = '50 - 75%'
    
    else:
        result = '75% - 100%'
    
    return result


df['checked_children'] = df.apply(check_children, axis = 1)
print(df['checked_children'].value_counts())

df['checked_family_status'] = df.apply(check_family_status, axis = 1)
print(df['checked_family_status'].value_counts())

df['checked_total_income'] = df.apply(check_total_income, axis = 1)
print(df['checked_total_income'].value_counts())

df.head()

### Вывод

Выделил словари для ответа на вопросы в шаге 3:
1. Возврат кредита в зависимости от детей
2. Семейное положение
3. Уровень дохода
4. Цели кредита - появились после лемматизации

### Шаг 3. Ответьте на вопросы <a id="step3"></a> 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
def answer_children(row):
    
    ratio = row[1] / row['sum']
    
    return ratio    


children_pivot = df.pivot_table(
    index = 'checked_children',
    columns = 'debt',
    values = 'children',
    aggfunc = 'count'
)

children_pivot['sum'] = children_pivot[0] + children_pivot[1]
children_pivot['ratio'] = children_pivot.apply(answer_children, axis = 1)

children_pivot

### Вывод

Наличие детей отрицательно сказывается на оплате кредита в срок.
- есть дети = 9.19%
- нет детей = 7.51%

Разница между ними составляет 1.68%. В результате семей с детьми меньше, чем без детей. Возможно, если бы количество семей с детьми было больше, то и процент неплательщиков был бы выше среди них. Дело в том, что дети - это дополнительные расходы, поэтмоу сложнее выплачивать кредит вовремя.

**Ответ: наличие детей уменьшает вероятность возврата кредита вовремя.**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
def answer_family_status(row):
    
    ratio = row[1] / row['sum']
    
    return ratio   


family_status_pivot = df.pivot_table(
    index = 'checked_family_status',
    columns = 'debt',
    values = 'family_status',
    aggfunc = 'count'
)

family_status_pivot['sum'] = family_status_pivot[0] + family_status_pivot[1]
family_status_pivot['ratio'] = family_status_pivot.apply(answer_family_status, axis = 1)

family_status_pivot

Особой разницы между семейными и холостяками практически нет: 
- холостяки = 8.49%
- семейные = 7.96% 

Разница между ними составляет < 1%.

**Ответ: семейное положение не влияет на возврат кредита вовремя.**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
def answer_total_income(row):
    
    ratio = row[1] / row['sum']
    
    return ratio   


total_income_pivot = df.pivot_table(
    index = 'checked_total_income',
    columns = 'debt',
    values = 'total_income',
    aggfunc = 'count'
)

total_income_pivot['sum'] = total_income_pivot[0] + total_income_pivot[1]
total_income_pivot['ratio'] = total_income_pivot.apply(answer_family_status, axis = 1)

total_income_pivot

### Вывод

Люди с доходом выше и ниже среднего чаще не платят кредит вовремя. Однако, люди из хвостов, с доходом сильно ниже среднего и сильно выше среднего, чаще платят вовремя. 

Cчитал: должники / все плательщики в этой категории.<br>

##### Результаты
- 0-25% = 7.69%<br>
- 25-50% = 8.36%<br>
- 50-75% = 8.37%<br>
- 75-100% = 6.86%

**Ответ: доход влияет на возврат кредита вовремя. Лучше выдавать кредит тем, у кого доход или сильно выше среднего, или сильно ниже среднего.**


- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_pivot = df.pivot_table(
    index = 'purpose_lemma',
    columns = 'debt',
    values = 'purpose',
    aggfunc = 'count'
)

purpose_pivot.columns = ['no_debt', 'has_debt']
purpose_pivot['sum'] = purpose_pivot['no_debt'] + purpose_pivot['has_debt']
purpose_pivot['ratio'] = (purpose_pivot['has_debt'] / (purpose_pivot['no_debt'] + purpose_pivot['has_debt'])).map('{:.2%}'.format)

purpose_pivot

### Вывод

##### Результаты
- займ на автомобиль = 9.33%<br>
- займ на недвижимость = 7.21%<br>
- займ на образование = 9.19%<br>
- займ на свадьбу = 7.92%

**Ответ: цель кредита влияет на возврат кредита вовремя. Чаще не возвращают кредит вовремя на автомобиль и образование. После свадьбы и при покупке недвижимости кредиты платятся вовремя. Возможно, дело в том, что на свадьбе дарят деньги, а по кредиту на недвижимость платежи делаются малыми частями на протяжении длительного периода времени.**

### Шаг 4. Общий вывод <a id="step4"></a> 

Есть показатели, которые влияют и не влияют на возврат кредита вовремя.

Интересным оказалось то, что среди плательщиков вовремя много людей с уровнем дохода ниже 25%. Вероятно, дело в размере кредита, который они берут. Скорее всего это маленькие суммы - их проще возвращать.

Дальше, что касается семейных пар и количества просрочернных кредитов на недвижимость. Можно предположить, что в семье появляется ребенок или еще один и появляется желание в увеличении жилплощади. Следовательно, семья берет кредит на жилье, но из-за расходов на ребенка не может платить вовремя.

Можно было бы выбрать семейных и недвижимость, но это как-нибудь в другой раз :) 

Если подытожить:
- наличие детей - влияет
- семейное положение - не влияет
- уровень дохода - влияет
- цель кредита - влияет

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.